[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/omar-merhebi/hlc-python/blob/master/Lesson_8d_RNAseq/Lesson_8d_RNAseq_Student_Version.ipynb)

# Lesson 8d - Basic RNAseq Analysis

### Learning objectives

Work through some more advanced data manipulation and analyses, working with a normalized RNAseq matrix and it's metadata.


In [282]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.stats import ttest_ind # for our t-test
from sklearn.decomposition import PCA # for our PCA

# setting an option in pandas to prevent some unnecessary warnings
pd.options.mode.chained_assignment = None 

Load the `meta_data.csv` file into a pandas data frame and store it in a variable called `metadata`.

In [293]:
# Load meta_data.csv as a pandas data frame


# Meta Data

Let's first view the metadata.

Here, we have unique sample identifiers in the `Sample` column. We can see the cell types that were used in the `Cell-type` column and there were two treatments applied, a `Control` treatment and a `Stress` treatment.

Without manually counting, let's report the number of samples we have for each Cell-type and Treatment combination.
We can do this using the `.value_counts()` function on the rows of our data frame we want to summarize.

In [ ]:
# use the value_counts() function to 


# Count Matrix
Now, let's load in our RNAseq count matrix as a pandas data frame and store it in a variable called `data`.

In [ ]:
# load count_matrix.csv as a pandas data frame and store in a variable called data


In this file, we have our sample identifiers along the column names and gene names along the rows.
How can we figure out how many genes we have in our data?

In [ ]:
# get the shape of the data


# PCA
Let's do a PCA to summarize the variation in our samples.
We are going to use the `PCA` function from the `sklearn.decomposition` package for this analysis. From looking at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) for the function, we can see that the PCA function wants data in the format samples x features. Our data is currently features x samples, so we'll have to transpose our matrix before doing the PCA.

In [ ]:
# transpose the data and store in a variable called t_data


Now that our data is in the correct format, lets run the PCA, storing the results in a variable called `pca_out` and look at the output.

In [ ]:
# run the PCA
pca = PCA(n_components=2) # specify the number of principal components
pca_out = pca.fit_transform(t_data) # run the analysis

pca_out

What data type did the pca give us?

We'll want to look at the PCA results in the context of the sample metadata, so let's add the PCA results we have to the metadata.

In [ ]:
metadata["PC1"] = pca_out[:, 0]
metadata["PC2"] = pca_out[:, 1]
metadata

Now, let's plot the PCA! We can use a scatter plot for this and let's use different colors and marker types to distinguish between the samples.

In [ ]:
# plot the PCA results, coloring by Treatment and styling the points by Cell-type
ax = sns.scatterplot(x="PC1", y="PC2", data=metadata, style="Cell-type", hue="Treatment")

# move the legend outside the plot area so we can see the data more clearly
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

It looks like one of our data points is separating pretty far away from the others on the PCA. Let's label the points to figure out which one it is.

In [ ]:
# Color the PCA by sample number to figure out which sample it is
ax = sns.scatterplot(x="PC1", y="PC2", data=metadata, style="Cell-type", hue="Treatment")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# Add labels to the pca points
for i in range(metadata.shape[0]):
    line = metadata.iloc[i, :]
    ax.text(line['PC1']+2, line['PC2'], str(line['Sample'])) 
    # the +2 above adjusts the x coord of the number label so it's not directly on the point

Let's say we talk to the person who ran the experiment and find out that there were significant issues with prepping that sample and we decide to exclude it from our analyses.

In [306]:
# Exclude sample 12 from the data and metadata
data = data.drop(["S12"], axis=1)
metadata = metadata[metadata['Sample'] != "S12"]=

Now, that we've excluded our outlier, let's rerun our PCA.

In [ ]:
# Copy the code from above to recalculate the PCA
pca = PCA(n_components=3) # specify the number of principal components
pca_out = pca.fit_transform(data.transpose()) # run the analysis

# Replace the PC1 and PC2 columns with our new analysis results
metadata["PC1"] = pca_out[:, 0]
metadata["PC2"] = pca_out[:, 1]

metadata

In [ ]:
ax = sns.scatterplot(x="PC1", y="PC2", data=metadata, style="Cell-type", hue="Treatment")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# Calculate Statistics
For the rest of our analysis, let's just look at the Bcells. Subset the data and metadata to just the Bcells. Store the subsetted dataframes in objects called `bcell_data` and `bcell_metadata`.

In [320]:
# subset metadata to only Bcells
bcell_metadata = metadata[metadata["Cell-type"] == "Bcells"]
# subset data to only Bcell samples
bcell_data = data[bcell_metadata["Sample"]]

Next, let's extract the sample ids for each treatment since that's the id that our data matrix uses.

In [259]:
# get the names of the samples in each treatment
control_samples = bcell_metadata[bcell_metadata["Treatment"] == "Control"]["Sample"]
stress_samples = bcell_metadata[bcell_metadata["Treatment"] == "Stress"]["Sample"]

Now we can use the sample ids we just extracted to isolate the data for each treatment and calculate the log2 fold change. Log2 Fold change can be calculated per gene like:
    log2( mean(treatment_data) / mean(control_data) )

In [ ]:
# calculate log2 fold change between control and stress
log2fc = np.log2( np.mean(bcell_data[stress_samples], axis=1) / np.mean(bcell_data[control_samples], axis=1) )

In [ ]:
bcell_data["log2FC"] = log2fc
bcell_data.head()

How many genes have a log2FC > 1 in our comparison?

How many genes have a log2FC < -1 in our comparison?

Lastly, let's test if the difference in the means between our two treatments is significant with a t-test.

In [ ]:
# initialize columns for pvalues
bcell_data["pval"] = None
bcell_data["neg_log10_pval"] = None

# loop through each gene
for gene in bcell_data.index:
    # get the expression data for each condition for the current gene
    stress_gene_data = bcell_data[stress_samples].loc[gene,:]
    control_gene_data = bcell_data[control_samples].loc[gene,:]

    # run a t-test and extract the p-value
    pval = ttest_ind(stress_gene_data, control_gene_data).pvalue
    
    # store the p-value in the pval column for that gene
    bcell_data.loc[gene,"pval"] = pval
    # calculate the -log10(pvalue) and store in the neg_log10_pval column
    bcell_data.loc[gene,"neg_log10_pval"] = -np.log10(pval)
    
bcell_data.head()

A common way to visualize this type of data is with a volcano plot, where Log2 fold change is on the x-axis and -Log10(pvalue) is on the y-axis. Let's use a scatter plot to make this.

In [ ]:
ax = sns.scatterplot(x="log2FC", y="neg_log10_pval", data=bcell_data)
ax

It'd be helpful if we could highlight which genes pass certain log2 fold change and/or pvalue thresholds. Let's build a logical statement to test if each gene is "significant" (in this case, has a log2FC > 1 OR a log2FC < 1 AND a pvalue < 0.05). 
Let's store the result in a column called "significant" in our bcell_data data frame so we can use it for plotting.

In [341]:
# conditional statement that returns T/F if gene passes thresholds


Let's make the same volcano plot, now coloring by significance.

In [ ]:
ax = sns.scatterplot(x="log2FC", y="neg_log10_pval", data=bcell_data, hue="significant", legend=False)
ax.set(xlabel='Log2FC', ylabel='-Log10(pval)', title='Differentially Expressed Genes Between\n Stress and Control in Bcells')

ax